### Pydantic
先安裝套件 ->pip install pydantic  
pydantic非常重視資料型別  
資料型態一定要有 **type hint**


In [1]:
import pydantic
pydantic.__version__

'2.10.1'

#### 三種建立實體的方法
##### 方法一:利用initial建立實體

In [6]:
class Person:
    def __init__(self,f_name:str,l_name:str,age:int):
        self.first_name = f_name
        self.last_name = l_name
        self.age = age

p1:Person = Person('John','Smith',age=42)
p1.first_name,p1.last_name,p1.age

('John', 'Smith', 42)

##### Pydantic 的核心是 BaseModel，所有模型都需要繼承自它

In [11]:
from pydantic import BaseModel
#繼承BaseModel
class Person(BaseModel):
    #field欄位
    first_name:str
    last_name:str
    age:int

#輸入的age若為字串，但因定義的age為數值，會自動轉換成數值並輸出
p1 = Person(first_name='John',last_name='Smith',age='42')
p1.first_name,p1.last_name,p1.age

('John', 'Smith', 42)

##### 若傳入的數據無法被驗證，會引發 ValidationError。

In [12]:
from pydantic import ValidationError
try:
    p1 = Person(first_name='John',last_name='Smith',age='42a')
except ValidationError as e:
    print(e)

1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='42a', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/int_parsing


##### 方法二:利用dict建立實體

In [13]:
dict_data:dict[str,any] = {
    'first_name':'John',
    'last_name':'Smith',
    'age':42
}
#使用class的方法model_validate
p2:Person = Person.model_validate(dict_data)
p2.first_name,p2.last_name,p2.age


('John', 'Smith', 42)

##### 方法三:使用json格式的字串建立實體

In [24]:
data_json:str = '''
{
"first_name":"John",
"last_name":"Smith",
"age":42
}
'''

p3 = Person.model_validate_json(data_json)
p3.first_name, p3.last_name, p3.age

('John', 'Smith', 42)

##### 沒有default值=require field，所以建立實體時，每個欄位都要設定值

In [26]:
class Person(BaseModel):
    #field欄位
    first_name:str
    last_name:str
    age:int

Person.model_validate({'age':42})

ValidationError: 2 validation errors for Person
first_name
  Field required [type=missing, input_value={'age': 42}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
last_name
  Field required [type=missing, input_value={'age': 42}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

##### 若有設定default值，建立實體時，不一定要設定值

In [29]:
class Person(BaseModel):
    #field欄位
    first_name:str
    last_name:str
    age:int = 0

p4 = Person.model_validate({
    'first_name':'John',
    'last_name':'Smith'
})

p4.age

0